In [40]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

## Cost matrix

In [41]:
#cost_matrix = pd.read_csv('../costs/costMatrixDistance.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial.csv', index_col=0)
cost_matrix = pd.read_csv('../costs/costMatrixDistance-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixDuration-2020.csv', index_col=0)
#cost_matrix = pd.read_csv('../costs/costMatrixFinancial-2020.csv', index_col=0)

# Convert the entire DataFrame to integers
cost_matrix = cost_matrix.astype(float)

# If the index of the DataFrame needs to be integers (e.g., if the index is non-numeric):
cost_matrix.index = cost_matrix.index.astype(str)

# If the column names need to be integers (if they are non-numeric or string-based):
cost_matrix.columns = cost_matrix.columns.astype(str)

In [42]:
cost_matrix

,Depot,ChIJwTC7A55ZwokRPNX9g7ngbaI,ChIJU1XImaNZwokRutunetC8XeE,ChIJG7L-TLVbwokRT36uIrwz2Mo,ChIJ5cPkuBtgwokRn55JgpGqjFA,ChIJ24V7r_31wokR-S71l2zqrwc,ChIJiUJ1DI5ZwokRWdK6SPg9BOY,ChIJYTxm0Gf2wokR5V0iFDQ-2x0,ChIJ_eQYpFdmwokR756MAH2tZPw,ChIJYzaRC2REwokRaH2rHpflSYk,...,ChIJReQHww5FwokRGlFWAkn4opg,ChIJQwtmq3v2wokRNOjx5cEBhXg,ChIJt2vTu1lawokRfVV3pBzItgU,ChIJX9DK2EBEwokRM9kDwjBVtag,ChIJmZKM98dgwokRZ6ZvctE2ImQ,ChIJ3wsoxKFZwokRRg8OvEjE0bc,ChIJQRpR6Jj0wokRl9myJ19eEvU,ChIJQYmJtx5awokR45_sopkXVNE,ChIJh4MG8h9awokR1kDLslu2C3E,ChIJbdnGmlxewokRobPWHJXQVjM
id,,,,,,,,,,,,,,,,,,,,,
Depot,0.908164,3.624313,3.064390,4.389933,10.402506,7.179099,0.970495,5.890938,17.795796,17.759441,...,13.875805,8.577781,5.444810,15.372453,11.601055,2.636302,13.965268,6.172048,6.172048,6.126072
ChIJwTC7A55ZwokRPNX9g7ngbaI,4.487928,0.692320,1.275166,3.404734,11.915880,6.139603,1.165866,6.603732,17.565902,17.655366,...,11.143219,8.364849,5.315607,15.963847,11.432615,0.907903,11.489765,1.901283,1.901283,7.486850
ChIJU1XImaNZwokRutunetC8XeE,5.226573,1.238316,0.693390,4.251927,12.064494,5.551015,1.546753,5.137780,17.167576,18.684990,...,13.796589,7.155312,5.823538,12.936463,11.820069,0.758407,14.100627,2.324565,2.324565,8.534553
ChIJG7L-TLVbwokRT36uIrwz2Mo,4.355250,2.956880,4.039189,0.696478,9.573396,10.271436,2.541625,11.223000,16.794486,4.019016,...,8.569997,11.704933,2.211792,11.801273,12.724804,4.125083,14.666276,2.171729,2.171729,6.119356
ChIJ5cPkuBtgwokRn55JgpGqjFA,13.477798,12.874358,10.590967,9.806480,0.996817,10.636402,11.766712,11.223698,9.554601,18.604800,...,6.510232,11.005756,6.830633,6.263304,8.757591,13.914388,12.551039,13.131675,13.131675,12.911588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ChIJ3wsoxKFZwokRRg8OvEjE0bc,4.795650,0.923201,0.723722,4.286476,11.202902,5.612903,1.651204,5.459457,16.934039,17.158424,...,11.603017,7.469938,6.504683,14.693145,12.141539,0.677820,13.880918,2.555090,2.555090,8.124578
ChIJQRpR6Jj0wokRl9myJ19eEvU,13.462167,7.422576,10.747481,11.828408,7.453325,7.339817,5.455323,5.742348,20.610216,15.243400,...,12.606605,4.035693,9.354276,13.006796,13.507891,11.292802,1.349578,11.045547,11.045547,6.673253
ChIJQYmJtx5awokR45_sopkXVNE,5.905736,2.135587,2.268411,2.207256,13.138740,9.151589,0.889933,6.893867,18.390186,18.863063,...,10.674825,9.968347,3.503519,12.979222,12.969834,2.693479,13.744055,0.726178,0.726178,9.728250


In [43]:
# Number of shops to visit
n = cost_matrix.shape[0]

# Calculate M
M = 0
for i in range(n):
    for j in range(n):
        M += cost_matrix.iloc[i,j]

# M cost for the path i->i
for i in range(n):
    cost_matrix.iloc[i,i] = M + 1

In [44]:
cost_matrix

,Depot,ChIJwTC7A55ZwokRPNX9g7ngbaI,ChIJU1XImaNZwokRutunetC8XeE,ChIJG7L-TLVbwokRT36uIrwz2Mo,ChIJ5cPkuBtgwokRn55JgpGqjFA,ChIJ24V7r_31wokR-S71l2zqrwc,ChIJiUJ1DI5ZwokRWdK6SPg9BOY,ChIJYTxm0Gf2wokR5V0iFDQ-2x0,ChIJ_eQYpFdmwokR756MAH2tZPw,ChIJYzaRC2REwokRaH2rHpflSYk,...,ChIJReQHww5FwokRGlFWAkn4opg,ChIJQwtmq3v2wokRNOjx5cEBhXg,ChIJt2vTu1lawokRfVV3pBzItgU,ChIJX9DK2EBEwokRM9kDwjBVtag,ChIJmZKM98dgwokRZ6ZvctE2ImQ,ChIJ3wsoxKFZwokRRg8OvEjE0bc,ChIJQRpR6Jj0wokRl9myJ19eEvU,ChIJQYmJtx5awokR45_sopkXVNE,ChIJh4MG8h9awokR1kDLslu2C3E,ChIJbdnGmlxewokRobPWHJXQVjM
id,,,,,,,,,,,,,,,,,,,,,
Depot,420532.611323,3.624313,3.064390,4.389933,10.402506,7.179099,0.970495,5.890938,17.795796,17.759441,...,13.875805,8.577781,5.444810,15.372453,11.601055,2.636302,13.965268,6.172048,6.172048,6.126072
ChIJwTC7A55ZwokRPNX9g7ngbaI,4.487928,420532.611323,1.275166,3.404734,11.915880,6.139603,1.165866,6.603732,17.565902,17.655366,...,11.143219,8.364849,5.315607,15.963847,11.432615,0.907903,11.489765,1.901283,1.901283,7.486850
ChIJU1XImaNZwokRutunetC8XeE,5.226573,1.238316,420532.611323,4.251927,12.064494,5.551015,1.546753,5.137780,17.167576,18.684990,...,13.796589,7.155312,5.823538,12.936463,11.820069,0.758407,14.100627,2.324565,2.324565,8.534553
ChIJG7L-TLVbwokRT36uIrwz2Mo,4.355250,2.956880,4.039189,420532.611323,9.573396,10.271436,2.541625,11.223000,16.794486,4.019016,...,8.569997,11.704933,2.211792,11.801273,12.724804,4.125083,14.666276,2.171729,2.171729,6.119356
ChIJ5cPkuBtgwokRn55JgpGqjFA,13.477798,12.874358,10.590967,9.806480,420532.611323,10.636402,11.766712,11.223698,9.554601,18.604800,...,6.510232,11.005756,6.830633,6.263304,8.757591,13.914388,12.551039,13.131675,13.131675,12.911588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ChIJ3wsoxKFZwokRRg8OvEjE0bc,4.795650,0.923201,0.723722,4.286476,11.202902,5.612903,1.651204,5.459457,16.934039,17.158424,...,11.603017,7.469938,6.504683,14.693145,12.141539,420532.611323,13.880918,2.555090,2.555090,8.124578
ChIJQRpR6Jj0wokRl9myJ19eEvU,13.462167,7.422576,10.747481,11.828408,7.453325,7.339817,5.455323,5.742348,20.610216,15.243400,...,12.606605,4.035693,9.354276,13.006796,13.507891,11.292802,420532.611323,11.045547,11.045547,6.673253
ChIJQYmJtx5awokR45_sopkXVNE,5.905736,2.135587,2.268411,2.207256,13.138740,9.151589,0.889933,6.893867,18.390186,18.863063,...,10.674825,9.968347,3.503519,12.979222,12.969834,2.693479,13.744055,420532.611323,0.726178,9.728250


## Shop demands

In [45]:
shop_demands = pd.read_csv('../2020_shop_locations.csv').set_index('id')[['stage', 'demand(kg)']].astype(float)
shop_demands = shop_demands.loc[shop_demands['stage'].isin([1]), 'demand(kg)'] # Choose the stages to visit
shops = shop_demands.index
nodes = shops.copy()
nodes = np.append('Depot', nodes)

## Closest neighbour heuristic

In [46]:
# Start from the storage location, go to the lowest cost shop, then go to the lowest cost shop from there, and so on until all shops are visited

# Create a list of visited shops
visited = ['Depot'] # Start from the storage location

# Create a list of unvisited shops
unvisited = shop_demands.index.to_list()

# Calculate the cost of the route
cost = 0

# Van capacity
van_capacity = 300
remaining_capacity = van_capacity

In [47]:
active_arcs = []  # List to store paths for all trucks
current_arcs = []  # Current path for the active truck

while len(unvisited) > 0:
    current = 'Depot'  # Start from the storage location
    while len(unvisited) >= 0:

        # Get the costs of the unvisited shops
        costs = cost_matrix.loc[current, unvisited]

        # Filter out shops that exceed the remaining capacity
        feasible_shops = [shop for shop in unvisited if shop_demands[shop] <= remaining_capacity]

        if not feasible_shops:
            # If no feasible shops, end the current truck's route and reset
            next_shop = 'Depot'  # Go back to the storage location
            current_arcs.append((current, next_shop))
            active_arcs.append(current_arcs)
            current_arcs = []  # Reset the path for the next truck
            remaining_capacity = van_capacity  # Reset the capacity for the new truck
            break  # Exit the loop to start with a new truck

        # Get the costs of the feasible shops
        feasible_costs = costs[costs.index.astype(str).isin(feasible_shops)]

        # Get the index of the lowest cost shop
        next_shop = feasible_costs.idxmin()

        # Store the arc between current and next_shop
        current_arcs.append((current, next_shop))

        # Add the lowest cost shop to the visited list
        visited.append(next_shop)

        # Remove the lowest cost shop from the unvisited list
        unvisited.remove(next_shop)

        # Add the cost of the route
        cost += feasible_costs.min()

        # Update the remaining capacity
        remaining_capacity -= shop_demands[next_shop]

        # Move to the next shop
        current = next_shop


# Cost and route

In [48]:
# Calculate the cost of the route
print('Cost of the route:', cost)
print('Route:', active_arcs)

Cost of the route: 85.42866668066573
Route: [[('Depot', 'ChIJiUJ1DI5ZwokRWdK6SPg9BOY'), ('ChIJiUJ1DI5ZwokRWdK6SPg9BOY', 'ChIJwTC7A55ZwokRPNX9g7ngbaI'), ('ChIJwTC7A55ZwokRPNX9g7ngbaI', 'ChIJU1XImaNZwokRutunetC8XeE'), ('ChIJU1XImaNZwokRutunetC8XeE', 'ChIJU3IVO-tZwokRx6iWrhKQTng'), ('ChIJU3IVO-tZwokRx6iWrhKQTng', 'ChIJG7L-TLVbwokRT36uIrwz2Mo'), ('ChIJG7L-TLVbwokRT36uIrwz2Mo', 'ChIJmZQsMgFbwokRMc5I7kPdQ7w'), ('ChIJmZQsMgFbwokRMc5I7kPdQ7w', 'ChIJG9cUYg_1wokRUNpvwM84Z4M'), ('ChIJG9cUYg_1wokRUNpvwM84Z4M', 'ChIJ2b9se6f1wokRUN8kWpFfzN0'), ('ChIJ2b9se6f1wokRUN8kWpFfzN0', 'ChIJT1ZEl4JYwokRXQMvGg9VMPE'), ('ChIJT1ZEl4JYwokRXQMvGg9VMPE', 'ChIJYTxm0Gf2wokR5V0iFDQ-2x0'), ('ChIJYTxm0Gf2wokR5V0iFDQ-2x0', 'ChIJ3R85DSb2wokR1XobETRMs5E'), ('ChIJ3R85DSb2wokR1XobETRMs5E', 'ChIJYzaRC2REwokRaH2rHpflSYk'), ('ChIJYzaRC2REwokRaH2rHpflSYk', 'ChIJ7-uIlEFEwokRlbHGtjrdFtM'), ('ChIJ7-uIlEFEwokRlbHGtjrdFtM', 'Depot')], [('Depot', 'ChIJ97BgiDtfwokRsbkP6Vl1seY'), ('ChIJ97BgiDtfwokRsbkP6Vl1seY', 'ChIJ24V7r_31wokR-S71l2zqr

In [49]:
def calculate_route_cost(routes, cost_matrix):
    """
    Calculate the total cost of routes by summing arc costs from the cost matrix.

    Parameters:
    - routes: List of routes, where each route is a list of (start, end) tuples
    - cost_matrix: DataFrame of transportation costs

    Returns:
    - Total route cost
    """
    total_cost = 0
    for route in routes:
        route_cost = sum(cost_matrix.loc[start, end] for start, end in route)
        total_cost += route_cost
    return total_cost

In [50]:
# Calculate cost of routes
original_routes_cost = calculate_route_cost(active_arcs, cost_matrix)

# Print detailed route cost information
print("Number of routes:", len(active_arcs))
print("Total route cost:", original_routes_cost)

Number of routes: 2
Total route cost: 105.32879023284076


In [51]:
# Flatten the list of arcs

flattened_arcs = [arc for route in active_arcs for arc in route]

flattened_arcs

[('Depot', 'ChIJiUJ1DI5ZwokRWdK6SPg9BOY'),
 ('ChIJiUJ1DI5ZwokRWdK6SPg9BOY', 'ChIJwTC7A55ZwokRPNX9g7ngbaI'),
 ('ChIJwTC7A55ZwokRPNX9g7ngbaI', 'ChIJU1XImaNZwokRutunetC8XeE'),
 ('ChIJU1XImaNZwokRutunetC8XeE', 'ChIJU3IVO-tZwokRx6iWrhKQTng'),
 ('ChIJU3IVO-tZwokRx6iWrhKQTng', 'ChIJG7L-TLVbwokRT36uIrwz2Mo'),
 ('ChIJG7L-TLVbwokRT36uIrwz2Mo', 'ChIJmZQsMgFbwokRMc5I7kPdQ7w'),
 ('ChIJmZQsMgFbwokRMc5I7kPdQ7w', 'ChIJG9cUYg_1wokRUNpvwM84Z4M'),
 ('ChIJG9cUYg_1wokRUNpvwM84Z4M', 'ChIJ2b9se6f1wokRUN8kWpFfzN0'),
 ('ChIJ2b9se6f1wokRUN8kWpFfzN0', 'ChIJT1ZEl4JYwokRXQMvGg9VMPE'),
 ('ChIJT1ZEl4JYwokRXQMvGg9VMPE', 'ChIJYTxm0Gf2wokR5V0iFDQ-2x0'),
 ('ChIJYTxm0Gf2wokR5V0iFDQ-2x0', 'ChIJ3R85DSb2wokR1XobETRMs5E'),
 ('ChIJ3R85DSb2wokR1XobETRMs5E', 'ChIJYzaRC2REwokRaH2rHpflSYk'),
 ('ChIJYzaRC2REwokRaH2rHpflSYk', 'ChIJ7-uIlEFEwokRlbHGtjrdFtM'),
 ('ChIJ7-uIlEFEwokRlbHGtjrdFtM', 'Depot'),
 ('Depot', 'ChIJ97BgiDtfwokRsbkP6Vl1seY'),
 ('ChIJ97BgiDtfwokRsbkP6Vl1seY', 'ChIJ24V7r_31wokR-S71l2zqrwc'),
 ('ChIJ24V7r_31wokR-S71l2z

In [52]:
# Convert the list of active arcs to a DataFrame
active_arcs_df = pd.DataFrame(flattened_arcs, columns=['From', 'To'])

# Save the DataFrame to a CSV file
active_arcs_df.to_csv('../routings/routing_closest_neighbor_2020_stage_1_Distance.csv', index=False)